Data source
https://github.com/vaishnavi-gawali/Analyzing-Sales-Data-using-Alteryx/tree/main/Datasets


# Import Library

In [1]:
import pandas as pd

c:\Users\isalo\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


# Data Preview

In [2]:
order_east = pd.read_excel("data/Customer Orders-East.xlsx")
order_west = pd.read_excel("data/Customer Orders-West.xlsx")\
    .rename(columns={'CustomerName': 'Name'})
order_details = pd.read_excel("data/Order Details.xlsx")
regions = pd.read_csv('data/Regions.csv')

In [3]:
order_east.head()

,Order ID,Order Date,State,Name
0,B-25757,"August 21, 2023",Florida,Reynolds
1,B-26003,"February 08, 2023",Florida,Dunn
2,B-25797,"September 30, 2023",Florida,Wong
3,B-25639,"April 27, 2023",Florida,Cooper
4,B-25681,"June 04, 2023",Florida,Hunter


In [4]:
order_west.head()

,Order ID,Order Date,Name,State
0,B-25935,"January 04, 2023",Lambert,Arizona
1,B-25653,"May 08, 2023",Morris,Arizona
2,B-25671,"May 27, 2023",Howard,Arizona
3,B-25743,"August 07, 2023",Evans,Arizona
4,B-25635,"April 26, 2023",Richardson,Arizona


In [5]:
order_details.head()

,Order ID,Sale Price,Cost per Item,Category,COD,Credit Card,Debit Card,EFT
0,B-25601,1275.0,18.14,Furniture - Bookcases,NaN,NaN,NaN,7.0
1,B-25601,66.0,15.60,Clothing - Scarf,NaN,NaN,NaN,5.0
2,B-25601,8.0,3.33,Clothing - Hat,3.0,NaN,NaN,NaN
3,B-25601,80.0,34.00,Electronics - Electronic Games,NaN,NaN,NaN,4.0
4,B-25602,119.0,15.50,Clothing - Dress,8.0,NaN,NaN,NaN


In [6]:
regions.head()

,State,Region
0,California,West
1,Florida,Southeast
2,New York,Northeast
3,Pennsylvania,Northeast
4,Illinois,Midwest


# Data Merge

In [7]:
order_east_with_region = pd.merge(order_east,regions, on='State')
order_west_with_region = pd.merge(order_west,regions, on='State')

# Add features

- Add Quantity column
- Split Category column into 2 new columns. 'Product Category' and 'Product Name'
- Drop Category column

In [8]:
order_no_details = pd.concat([order_east_with_region, order_west_with_region]).reset_index(drop=True)

all_order = pd.merge(order_no_details, order_details, on='Order ID')
all_order['Quantity'] = all_order[['COD',"Credit Card","Debit Card","EFT"]].sum(axis=1)

split_product = all_order['Category'].str.split(' - ', expand=True)
split_product.fillna('', inplace=True)
split_product[1] = (split_product[1] + ' ' + split_product[2])
split_product.drop(columns=[2], inplace=True)

all_order[['Product Category', 'Product Name']] = split_product
all_order.drop(columns=['Category'], inplace=True)

In [9]:
all_order

,Order ID,Order Date,State,Name,Region,Sale Price,Cost per Item,COD,Credit Card,Debit Card,EFT,Quantity,Product Category,Product Name
0,B-25757,"August 21, 2023",Florida,Reynolds,Southeast,106.0,13.00,7.0,NaN,NaN,NaN,7.0,Clothing,Hat
1,B-25757,"August 21, 2023",Florida,Reynolds,Southeast,14.0,9.00,NaN,1.0,NaN,NaN,1.0,Clothing,Hat
2,B-25757,"August 21, 2023",Florida,Reynolds,Southeast,165.0,45.00,NaN,3.0,NaN,NaN,3.0,Clothing,Scarf
3,B-25757,"August 21, 2023",Florida,Reynolds,Southeast,17.0,3.33,NaN,NaN,NaN,3.0,3.0,Clothing,Hat
4,B-25757,"August 21, 2023",Florida,Reynolds,Southeast,17.0,7.50,NaN,NaN,4.0,NaN,4.0,Clothing,Skirt
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1582,B-26008,"February 09, 2023",Wisconsin,Elliott,Midwest,22.0,18.00,NaN,1.0,NaN,NaN,1.0,Clothing,Scarf
1583,B-26008,"February 09, 2023",Wisconsin,Elliott,Midwest,57.0,6.60,NaN,5.0,NaN,NaN,5.0,Clothing,Leggings
1584,B-25860,"November 15, 2023",Wisconsin,Thomas,Midwest,112.0,29.33,NaN,3.0,NaN,NaN,3.0,Clothing,Shirt
1585,B-25716,"July 11, 2023",Wisconsin,Campbell,Midwest,58.0,14.50,NaN,NaN,NaN,4.0,4.0,Clothing,Dress


# Export Processed Data to Excel

In [16]:
all_order.to_excel("all_order.xlsx")